In [1]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [2]:
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_SVR-MLP_Model_Day.csv'

In [3]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 368849 entries, 1 to 368849
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_MEDICAO_HORA         368849 non-null  object 
 1   PRECIPITACAO            368849 non-null  float64
 2   PRESSAO_ATMOSFERICA     368849 non-null  float64
 3   TEMPERATURA_DO_AR_C     368849 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  368849 non-null  float64
 5   VELOCIDADE_VENTO        368849 non-null  float64
 6   SK_PONTO                368849 non-null  int64  
 7   VL_MEDICAO              368849 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 25.3+ MB


In [5]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [6]:
def salvar_resultado(sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  data = [sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration]
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)   

In [7]:
def imprime_grafico(y_test, predict):
  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  npredicts=1000
  pyplot.figure(figsize=(10,7)) 
  pyplot.scatter(np.arange(0,y_test.shape[0],1)[0:npredicts],np.array(y_test)[0:npredicts], color = 'black')
  pyplot.plot(np.arange(0,predict.shape[0],1)[0:npredicts], np.array(predict)[0:npredicts], color = 'blue', linewidth=2)
  pyplot.xlabel('n seq.')
  pyplot.ylabel('consumo l/h')
  pyplot.show()

In [8]:
#crop date time to date 
dataset['DATA']=dataset['DT_MEDICAO_HORA'].str.split(' ').str[0]
dataset.drop(['DT_MEDICAO_HORA'],axis=1, inplace=True)

# Grouping data for Day
# #cols = 'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'
dataset = dataset.groupby(['SK_PONTO','DATA'], as_index=False).agg({'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'})
#df_result.rename(columns=cols)

In [9]:
def previsao_SVR(sk_ponto, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['VL_MEDICAO'].shift(n_step)  
    
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 6 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 6 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   
  
  #Estimate Et for treinam and test
  # For treinam data
  Lt_train = grid.predict(X_train)
  Yt_train = y_train
  Et_train = Yt_train - Lt_train  

  # For test data
  Lt_test = predict
  Yt_test = y_test
  Et_test = Yt_test - Lt_test  

  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_train, columns=['VL_Et'])  

  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)    

  #drop rows with null values
  df_Et.dropna(inplace=True)  

  Nt_train = df_Et['VL_Et']  

  df_Et.drop(['VL_Et'],axis=1, inplace=True)  

  X_train = df_Et
  y_train = Nt_train
  
  # MLP model for hybrid prediction for non linear component Et_train 

  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(X_train.shape[0] * 0.75)) # 
    
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((X_train.shape[0] - nlinhas) / 5))
  size_split = 5
    
  # Cross-validated for time series
  ts_cv_mlp = TimeSeriesSplit(
        n_splits=size_split, # Number of divisions
        max_train_size=max_size_train_split,    # maximum size of each set. of training
        gap=2, # number of samples to exclude between each training and testing set
        test_size=max_size__test_split, # maximum size of each set. of test.
  )  

  param_grid = {
       'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
       'max_iter': [ 500], # maximum iterations
       'activation': [ 'relu','identity'], # activation function
       'solver': ['adam'], # weight optimization algorithm
       'alpha': [0.0001, 0.001, 0.01],  # alpha strength of regularization
  }
    
  modelo = MLPRegressor(random_state=0)
    
  grid_mlp = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv_mlp, verbose=1)
  grid_mlp.fit(np.array(X_train), np.array(y_train))  

  resultado = str(grid.best_params_)
  
  #MAPE to Et predicted on MLP Model
  predict=grid_mlp.predict(X_train)  
  # Recomposition of Nt + Lt form train data
  Lt_train2 = Lt_train[n_time_steps:len(Lt_train)]
  Yt_train2 = Yt_train[n_time_steps:len(Yt_train)]
  Yt_train_predicted = predict + Lt_train2
  ((np.mean(np.abs(Yt_train2 - Yt_train_predicted) / (Yt_train2)))) * 100
  
  # For test data
  #Lt_test = predict
  #Yt_test = y_test
  #Et_test = Yt_test - Lt_test
  
  
  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_test, columns=['VL_Et'])
  
  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)  
  
  #drop rows with null values
  df_Et.dropna(inplace=True)
  
  Nt_test = df_Et['VL_Et']
  
  df_Et.drop(['VL_Et'],axis=1, inplace=True)
  
  X_test = df_Et
  y_test = Nt_test
  
  predict=grid_mlp.predict(X_test)  
  
  # Recomposition of Nt + Lt form test data
  Lt_test2 = Lt_test[n_time_steps:len(Lt_test)]
  Yt_test2 = Yt_test[n_time_steps:len(Yt_test)]
  Yt_test_predicted = predict + Lt_test2
  MAPE = ((np.mean(np.abs(Yt_test2 - Yt_test_predicted) / (Yt_test2)))) * 100
  print('MAPE',MAPE)
  
  predict = Yt_test_predicted
  y_test = Yt_test2
  
  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)   

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  #imprime_grafico(Yt_train2, Yt_train_predicted)

  salvar_resultado(sk_ponto, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, Duracao)

In [10]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    for n_time_steps in range(1,7): #predict with 1 to 6 past values of medition
        previsao_SVR(sk, df_ponto, n_time_steps)

forecast for sk_ponto =  1
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


MAPE 29.038663120005648
['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[1, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-08, 'kernel': 'rbf'}", 1, 5282729.393320082, 2298.4188898719226, 667.4617601184491, 29.038663120005648, 55.61105179786682]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


...

c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


MAPE 110.00833720977563
['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[13, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}", 5, 820892457.4921737, 28651.220872629034, 6995.197214727326, 110.00833720977563, 50.17323446273804]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Fitting 5 folds for each of 24 candidates, totalling 120 fits
MAPE 110.35744043121537
['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[13, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-06, 'kernel': 'rbf'}", 6, 824079148.1810704, 28706.77878447999, 7035.406048095945, 110.35744043121537, 46.45367121696472]


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
